# Benodigde libraries importeren

In [255]:
import pandas as pd
from datetime import datetime
import numpy as np
import pandas.api.types as ptypes


# Bestand inladen en data filteren

In [256]:
# excel bestand maxilia geimporteerd en kolommen gefilterd 

df = pd.read_csv('expdata 20190912 1002.csv', sep = None, delimiter = None, usecols = ['email','Orderdate','OrderID','TurnoverLead', 'BuyinLead', 'LeadPhase','LeadStatus'], engine = 'python') 
df.head()

,OrderID,Orderdate,email,TurnoverLead,BuyinLead,LeadPhase,LeadStatus
0,27654,22-4-2014,janneke.van.wingerden@hotmail.com,840.0,0.0,Order,Gefactureerd
1,30777,9-4-2015,marloessanders@izit.nl,300.0,0.0,Order,Gefactureerd
2,32291,30-7-2014,info@ferox-design.com,235.0,0.0,Order,Gefactureerd
3,32291,30-7-2014,info@ferox-design.com,235.0,0.0,Order,Gefactureerd
4,33824,27-5-2016,johan@thermservice.be,0.0,0.0,Order,Gefactureerd


In [257]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273322 entries, 0 to 273321
Data columns (total 7 columns):
OrderID         273322 non-null int64
Orderdate       131183 non-null object
email           272921 non-null object
TurnoverLead    273322 non-null float64
BuyinLead       273322 non-null float64
LeadPhase       273322 non-null object
LeadStatus      271268 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 14.6+ MB


In [258]:
# lege rijen 'NaT' eruit gefilterd 
# Email bevat @ controle
# Duplicaten verwijderen in orderID

df = df.dropna(axis=0, how = 'any', inplace=False)
df = df[~df['email'].str.contains('@')==False]
df = df.drop_duplicates('OrderID', keep='first')
df.head()

,OrderID,Orderdate,email,TurnoverLead,BuyinLead,LeadPhase,LeadStatus
0,27654,22-4-2014,janneke.van.wingerden@hotmail.com,840.0,0.0,Order,Gefactureerd
1,30777,9-4-2015,marloessanders@izit.nl,300.0,0.0,Order,Gefactureerd
2,32291,30-7-2014,info@ferox-design.com,235.0,0.0,Order,Gefactureerd
4,33824,27-5-2016,johan@thermservice.be,0.0,0.0,Order,Gefactureerd
5,34406,4-2-2015,miranda@dynadental.com,1185.0,0.0,Order,Gefactureerd


In [259]:
# Orderdate to datetime. Vul juiste date format in
df['Orderdate']=pd.to_datetime(df['Orderdate'], format='%d-%m-%Y')


In [260]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106583 entries, 0 to 273321
Data columns (total 7 columns):
OrderID         106583 non-null int64
Orderdate       106583 non-null datetime64[ns]
email           106583 non-null object
TurnoverLead    106583 non-null float64
BuyinLead       106583 non-null float64
LeadPhase       106583 non-null object
LeadStatus      106583 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.5+ MB


In [261]:
df.describe()

,OrderID,TurnoverLead,BuyinLead
count,106583.000000,106583.000000,106583.00000
mean,147886.233912,611.979496,360.30912
std,61013.462495,1391.539161,1031.11294
min,27654.000000,-392.000000,-1001.51000
25%,94114.000000,167.000000,34.50000
50%,148056.000000,300.400000,151.50000
75%,201786.500000,610.000000,358.47000
max,252808.000000,174519.500000,141338.84000


In [262]:
# Data filteren (specifiek voor Maxilia)
# Leadphase = order
# Leadstatus = gefactureerd
df = df[~df['LeadPhase'].str.contains('Order')==False]
df = df[~df['LeadStatus'].str.contains('Gefactureerd')==False]
df.head()

,OrderID,Orderdate,email,TurnoverLead,BuyinLead,LeadPhase,LeadStatus
0,27654,2014-04-22,janneke.van.wingerden@hotmail.com,840.0,0.0,Order,Gefactureerd
1,30777,2015-04-09,marloessanders@izit.nl,300.0,0.0,Order,Gefactureerd
2,32291,2014-07-30,info@ferox-design.com,235.0,0.0,Order,Gefactureerd
4,33824,2016-05-27,johan@thermservice.be,0.0,0.0,Order,Gefactureerd
5,34406,2015-02-04,miranda@dynadental.com,1185.0,0.0,Order,Gefactureerd


In [263]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97825 entries, 0 to 272752
Data columns (total 7 columns):
OrderID         97825 non-null int64
Orderdate       97825 non-null datetime64[ns]
email           97825 non-null object
TurnoverLead    97825 non-null float64
BuyinLead       97825 non-null float64
LeadPhase       97825 non-null object
LeadStatus      97825 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.0+ MB


In [264]:
# Check data voldoet aan bepaalde condities
assert ptypes.is_datetime64_any_dtype(df['Orderdate'])
assert df['email'].str.contains('@').all()
assert df['LeadPhase'].eq('Order').all()
assert df['LeadStatus'].eq('Gefactureerd').all()


# Data grouperen per klant

In [265]:
# group by email (customer)
# hoeveel orders zijn er gedaan per customer
# totale uitgaven per customer 

df_clv = df.groupby('email').agg({
                                                        'Orderdate': lambda date: (date.max() - date.min()).days,
                                                        'OrderID': lambda num: len(num), 
                                                        'TurnoverLead': lambda price: price.sum()
                                                    }) 
df_clv.head()

,Orderdate,OrderID,TurnoverLead
email,,,
03vichy@gmail.com,0,1,259.0
06alexisbrunet@gmail.com,0,1,180.0
070paulus070@gmail.com,0,1,33.5
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDEF.NL,0,1,1230.5
1.vorsitzende.gv.muenster@gmail.com,0,1,344.5


In [266]:
# rename kolommen 

df_clv.columns = ['days_customer', 'total_transactions', 'total_spent']
df_clv.head()

,days_customer,total_transactions,total_spent
email,,,
03vichy@gmail.com,0,1,259.0
06alexisbrunet@gmail.com,0,1,180.0
070paulus070@gmail.com,0,1,33.5
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDEF.NL,0,1,1230.5
1.vorsitzende.gv.muenster@gmail.com,0,1,344.5


In [267]:
df_clv.describe()

,days_customer,total_transactions,total_spent
count,67777.000000,67777.000000,67777.000000
mean,76.580684,1.443336,886.926759
std,225.263499,1.724549,2695.224340
min,0.000000,1.000000,-392.000000
25%,0.000000,1.000000,173.000000
50%,0.000000,1.000000,332.000000
75%,0.000000,1.000000,757.600000
max,1686.000000,112.000000,167285.400000


# Berekenen van de totale CLV

In [268]:
# gemiddelde order waarde 

avg_order_value = sum(df_clv['total_spent']) / sum(df_clv['total_transactions'])
print(avg_order_value)

614.4976738052648


In [269]:
# hoe vaak koopt iemand gemiddeld iets

purchase_frequency = sum(df_clv['total_transactions']) / df_clv.shape[0]
print(purchase_frequency)

1.4433362350059755


In [270]:
# hoe vaak komt iemand terug

repeat_rate = df_clv[df_clv.total_transactions > 1].shape[0] / df_clv.shape[0] 
print(repeat_rate)

0.20118919397435708


In [271]:
# churnrate: hoeveel % doet na 1 aankoop geen bestelling meer

churn_rate = 1 - repeat_rate
print(churn_rate)

0.7988108060256429


In [272]:
# gemiddelde marge  

profit = sum(df['TurnoverLead']) - sum(df['BuyinLead'])
margin = profit / sum(df['TurnoverLead'])
    
print(margin)

0.4098393753820391


In [273]:
# totale clv & clv marge 
# in het blok hierboven is de marge berekend ('margin'), maar je kan ook een getal noteren ipv 'margin'

total_clv = (avg_order_value * purchase_frequency) / churn_rate 
total_clv_margin = total_clv * margin

print(total_clv) # het eerste getal is de totale clv
print(total_clv_margin) # het tweede getal is de clv waarbij rekening is gehouden met de berekende marge

1110.3089145010256
455.04831200021016


In [274]:
# Bovenstaande cijfers in kolom neerzetten zodat de data in datastudio kan worden gebruikt
set_total_clv = avg_order_value, purchase_frequency, churn_rate, total_clv, total_clv_margin 
df_total_clv = pd.DataFrame(set_total_clv, columns= ['Waarde'], index = ['Gemiddelde orderwaarde' , 'Frequentie aankoop', 'Churn rate' , 'CLV', 'CLV marge'])
df_clv_values=df_total_clv.T.groupby(level=0).agg(lambda x : x.values.tolist()).stack().apply(pd.Series).unstack().sort_index(level=1,axis=1)
df_clv_values.columns=df_clv_values.columns.droplevel(level=0)
df_clv_values.head()

,CLV,CLV marge,Churn rate,Frequentie aankoop,Gemiddelde orderwaarde
Waarde,1110.308915,455.048312,0.798811,1.443336,614.497674


In [275]:
df_clv_values.to_excel('CLV_waardes.xlsx')

   # Churn rate per maand uiteenzetten

In [276]:
# transacties per klant per maand uiteenzetten
df_month = df.set_index('Orderdate')
df_monthly = df_month.pivot_table(
    index=['email'],
    columns=pd.Grouper(freq='M'),
    values='OrderID', 
    aggfunc='count',
    fill_value=0
)

df_monthly.head()

Orderdate,2014-04-30 00:00:00,2014-07-31 00:00:00,2014-09-30 00:00:00,2014-10-31 00:00:00,2014-11-30 00:00:00,2014-12-31 00:00:00,2015-01-31 00:00:00,2015-02-28 00:00:00,2015-03-31 00:00:00,2015-04-30 00:00:00,...,2018-12-31 00:00:00,2019-01-31 00:00:00,2019-02-28 00:00:00,2019-03-31 00:00:00,2019-04-30 00:00:00,2019-05-31 00:00:00,2019-06-30 00:00:00,2019-07-31 00:00:00,2019-08-31 00:00:00,2019-09-30 00:00:00
email,,,,,,,,,,,,,,,,,,,,,
03vichy@gmail.com,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
06alexisbrunet@gmail.com,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
070paulus070@gmail.com,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDEF.NL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.vorsitzende.gv.muenster@gmail.com,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [277]:
repeat_rate = df_monthly[df_monthly > 1].count() / df_monthly[df_monthly > 0].count()
df_churn = repeat_rate.to_frame(name='repeat_rate')

In [278]:
df_churn['churn_rate'] = 1 - df_churn['repeat_rate']
df_churn.head()


,repeat_rate,churn_rate
Orderdate,,
2014-04-30,0.0,1.0
2014-07-31,0.0,1.0
2014-09-30,0.0,1.0
2014-10-31,0.0,1.0
2014-11-30,0.0,1.0


# Churn per 6 maanden en per jaar

In [279]:
# Churn per 6 maanden
df_6_months = df_month.pivot_table(
    index=['email'],
    columns=pd.Grouper(freq='6M'),
    values='OrderID', 
    aggfunc='count',
    fill_value=0
)

df_6_months.head()

Orderdate,2014-04-30 00:00:00,2014-10-31 00:00:00,2015-04-30 00:00:00,2015-10-31 00:00:00,2016-04-30 00:00:00,2016-10-31 00:00:00,2017-04-30 00:00:00,2017-10-31 00:00:00,2018-04-30 00:00:00,2018-10-31 00:00:00,2019-04-30 00:00:00,2019-10-31 00:00:00
email,,,,,,,,,,,,
03vichy@gmail.com,0,0,0,0,0,0,0,0,0,0,1,0
06alexisbrunet@gmail.com,0,0,0,0,0,0,0,0,1,0,0,0
070paulus070@gmail.com,0,0,0,0,0,0,0,0,0,0,0,1
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDEF.NL,0,0,0,0,0,1,0,0,0,0,0,0
1.vorsitzende.gv.muenster@gmail.com,0,0,0,0,0,0,0,0,0,1,0,0


In [280]:
repeat_rate = df_6_months[df_6_months > 1].count() / df_6_months[df_6_months > 0].count()
df_churn_6_months = repeat_rate.to_frame(name='repeat_rate_6_months')

In [281]:
df_churn_6_months['churn_rate_6_months'] = 1 - df_churn_6_months['repeat_rate_6_months']
df_churn_6_months

,repeat_rate_6_months,churn_rate_6_months
Orderdate,,
2014-04-30,0.000000,1.000000
2014-10-31,0.000000,1.000000
2015-04-30,0.063094,0.936906
2015-10-31,0.086375,0.913625
2016-04-30,0.090199,0.909801
2016-10-31,0.102916,0.897084
2017-04-30,0.110682,0.889318
2017-10-31,0.124986,0.875014
2018-04-30,0.110779,0.889221


In [282]:
# Churn per jaar
df_year = df_month.pivot_table(
    index=['email'],
    columns=pd.Grouper(freq='Y'),
    values='OrderID', 
    aggfunc='count',
    fill_value=0
)

df_year.head()

Orderdate,2014-12-31 00:00:00,2015-12-31 00:00:00,2016-12-31 00:00:00,2017-12-31 00:00:00,2018-12-31 00:00:00,2019-12-31 00:00:00
email,,,,,,
03vichy@gmail.com,0,0,0,0,1,0
06alexisbrunet@gmail.com,0,0,0,0,1,0
070paulus070@gmail.com,0,0,0,0,0,1
1.NLD.MINUSMA.GAO.AHDET.COGP.S1@B-SC1.TTN.MINDEF.NL,0,0,1,0,0,0
1.vorsitzende.gv.muenster@gmail.com,0,0,0,0,1,0


In [283]:
repeat_rate = df_year[df_year > 1].count() / df_year[df_year > 0].count()
df_churn_year = repeat_rate.to_frame(name='repeat_rate_year')

In [284]:
df_churn_year['churn_rate_year'] = 1 - df_churn_year['repeat_rate_year']
df_churn_year

,repeat_rate_year,churn_rate_year
Orderdate,,
2014-12-31,0.000000,1.000000
2015-12-31,0.114538,0.885462
2016-12-31,0.130083,0.869917
2017-12-31,0.151944,0.848056
2018-12-31,0.148535,0.851465
2019-12-31,0.131676,0.868324


# Transacties, uitgaves en unieke klanten per maand

In [285]:
# set orderdate als index
df.set_index('Orderdate', inplace=True)

In [286]:
# groepeer totale transacties / opbrengst per maand / unieke klanten per maand

df_transacties = df.groupby(pd.Grouper(freq='M')).agg(
    {
        'OrderID': len, 
        'TurnoverLead': sum,
        'email': pd.Series.nunique
    }
)

df_transacties.head()

,OrderID,TurnoverLead,email
Orderdate,,,
2014-04-30,1,840.0,1
2014-05-31,0,0.0,0
2014-06-30,0,0.0,0
2014-07-31,1,235.0,1
2014-08-31,0,0.0,0


In [287]:
df_transacties.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 66 entries, 2014-04-30 to 2019-09-30
Freq: M
Data columns (total 3 columns):
OrderID         66 non-null int64
TurnoverLead    66 non-null float64
email           66 non-null int64
dtypes: float64(1), int64(2)
memory usage: 4.6 KB


In [288]:
# Hernoem kolommen 
df_transacties.columns = ['total_transactions', 'total_spent', 'total_unique_customers']
df_transacties.head(5)

,total_transactions,total_spent,total_unique_customers
Orderdate,,,
2014-04-30,1,840.0,1
2014-05-31,0,0.0,0
2014-06-30,0,0.0,0
2014-07-31,1,235.0,1
2014-08-31,0,0.0,0


In [289]:
df_transacties.describe()

,total_transactions,total_spent,total_unique_customers
count,66.000000,6.600000e+01,66.000000
mean,1482.196970,9.108066e+05,1393.166667
std,922.956741,5.712635e+05,856.845980
min,0.000000,0.000000e+00,0.000000
25%,897.250000,4.805995e+05,871.250000
50%,1395.500000,9.146664e+05,1314.500000
75%,2106.250000,1.291857e+06,1955.750000
max,3810.000000,2.391474e+06,3538.000000


In [290]:
df_transacties.to_excel('Overzicht_per_maand.xlsx')

# CLV per maand berekenen met churn per maand

In [291]:
# CLV per maand berekenen
# margin is nu over gehele dataset, je kunt deze ook nog per maand uitrekenen
df_transacties['purchase_frequency_month'] = df_transacties['total_transactions'] / df_transacties['total_unique_customers']
df_transacties['avg_order_value_month'] = df_transacties['total_spent'] / df_transacties['total_transactions'] 
df_transacties['CLV'] = (df_transacties['avg_order_value_month'] * df_transacties['purchase_frequency_month']) / df_churn['churn_rate']
df_transacties['CLV_margin'] = df_transacties['CLV'] * margin
df_transacties.head()

,total_transactions,total_spent,total_unique_customers,purchase_frequency_month,avg_order_value_month,CLV,CLV_margin
Orderdate,,,,,,,
2014-04-30,1,840.0,1,1.0,840.0,840.0,344.265075
2014-05-31,0,0.0,0,NaN,NaN,NaN,NaN
2014-06-30,0,0.0,0,NaN,NaN,NaN,NaN
2014-07-31,1,235.0,1,1.0,235.0,235.0,96.312253
2014-08-31,0,0.0,0,NaN,NaN,NaN,NaN


In [292]:
# nieuwe dataset maken om gemiddelde CLV te berekenen
# Drop lege kolommen 
# Toevoegen index kolom om gemiddelde mee te berekenen 
df_clv_cum= df_transacties[['CLV', 'CLV_margin']].copy()
df_clv_cum.dropna(inplace=True)
df_clv_cum['Index'] = np.arange(1, len(df_clv_cum) + 1)
df_clv_cum['CLV_AVG'] = df_clv_cum['CLV'].cumsum().div(df_clv_cum['Index'])
df_clv_cum['CLV_margin_AVG'] = df_clv_cum['CLV_margin'].cumsum().div(df_clv_cum['Index'])
df_clv_cum.head()

,CLV,CLV_margin,Index,CLV_AVG,CLV_margin_AVG
Orderdate,,,,,
2014-04-30,840.000000,344.265075,1,840.000000,344.265075
2014-07-31,235.000000,96.312253,2,537.500000,220.288664
2014-09-30,219.000000,89.754823,3,431.333333,176.777384
2014-10-31,564.000000,231.149408,4,464.500000,190.370390
2014-11-30,675.251111,276.744494,5,506.650222,207.645211


# CLV per maand berekenen met churn per 6 maanden

In [293]:
# Code om churn rate over 6 maanden en jaar bij juiste maanden en jaar vinden
# Zonder deze code komt de churn rate over 6 maanden alleen bij de laatste maand uit en over een jaar alleen bij december
# Deze code vult de data daarvoor aan met de berekende churn rate over deze periode
def find_churn_rate_6_months(date):
    """
    Iterate through the 'df_churn_6_months' dataframe.
    When the date is smaller than the date on the index, 
    return the corresponding churn rate 
    Eg: July < Oct, so return October's churn rate
    """
    for m in df_churn_6_months['churn_rate_6_months'].iteritems():
        if date <= m[0]:
            return m[1]
        
def find_churn_rate_1_year(date):
    """
    Iterate through the 'df_churn_year' dataframe.
    When the date is smaller than the year on the index, 
    return the corresponding churn rate 
    """
    for y in df_churn_year['churn_rate_year'].iteritems():
        if date <= y[0]:
            return y[1]

In [294]:
# CLV berekenen met churn per 6 maanden
# Eerste regel zoekt de juiste churn rate bij de juiste maanden met behulp van bovenstaande code
df_transacties['churn_rate_6_months'] = df_transacties.index.to_series().apply(lambda x: find_churn_rate_6_months(x))
df_transacties['CLV_churn_6M'] = (df_transacties['avg_order_value_month'] * df_transacties['purchase_frequency_month']) / df_transacties['churn_rate_6_months']
df_transacties['CLV_margin_churn_6M'] = df_transacties['CLV_churn_6M'] * margin
df_transacties.head(10)

,total_transactions,total_spent,total_unique_customers,purchase_frequency_month,avg_order_value_month,CLV,CLV_margin,churn_rate_6_months,CLV_churn_6M,CLV_margin_churn_6M
Orderdate,,,,,,,,,,
2014-04-30,1,840.00,1,1.000000,840.000000,840.000000,344.265075,1.000000,840.000000,344.265075
2014-05-31,0,0.00,0,NaN,NaN,NaN,NaN,1.000000,NaN,NaN
2014-06-30,0,0.00,0,NaN,NaN,NaN,NaN,1.000000,NaN,NaN
2014-07-31,1,235.00,1,1.000000,235.000000,235.000000,96.312253,1.000000,235.000000,96.312253
2014-08-31,0,0.00,0,NaN,NaN,NaN,NaN,1.000000,NaN,NaN
2014-09-30,1,219.00,1,1.000000,219.000000,219.000000,89.754823,1.000000,219.000000,89.754823
2014-10-31,3,1692.00,3,1.000000,564.000000,564.000000,231.149408,1.000000,564.000000,231.149408
2014-11-30,9,6077.26,9,1.000000,675.251111,675.251111,276.744494,0.936906,720.724213,295.381161
2014-12-31,57,56038.15,57,1.000000,983.125439,983.125439,402.923516,0.936906,1049.331569,430.057395


# CLV per maand berekenen met churn per jaar

In [295]:
# CLV per maand berekenen met churn berekend over een jaar
df_transacties['churn_rate_1_year'] = df_transacties.index.to_series().apply(lambda x: find_churn_rate_1_year(x))
df_transacties['CLV_churn_Y'] = (df_transacties['avg_order_value_month'] * df_transacties['purchase_frequency_month']) / df_transacties['churn_rate_1_year']
df_transacties['CLV_margin_churn_Y'] = df_transacties['CLV_churn_Y'] * margin
df_transacties.head(10)

,total_transactions,total_spent,total_unique_customers,purchase_frequency_month,avg_order_value_month,CLV,CLV_margin,churn_rate_6_months,CLV_churn_6M,CLV_margin_churn_6M,churn_rate_1_year,CLV_churn_Y,CLV_margin_churn_Y
Orderdate,,,,,,,,,,,,,
2014-04-30,1,840.00,1,1.000000,840.000000,840.000000,344.265075,1.000000,840.000000,344.265075,1.000000,840.000000,344.265075
2014-05-31,0,0.00,0,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN
2014-06-30,0,0.00,0,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN
2014-07-31,1,235.00,1,1.000000,235.000000,235.000000,96.312253,1.000000,235.000000,96.312253,1.000000,235.000000,96.312253
2014-08-31,0,0.00,0,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN
2014-09-30,1,219.00,1,1.000000,219.000000,219.000000,89.754823,1.000000,219.000000,89.754823,1.000000,219.000000,89.754823
2014-10-31,3,1692.00,3,1.000000,564.000000,564.000000,231.149408,1.000000,564.000000,231.149408,1.000000,564.000000,231.149408
2014-11-30,9,6077.26,9,1.000000,675.251111,675.251111,276.744494,0.936906,720.724213,295.381161,1.000000,675.251111,276.744494
2014-12-31,57,56038.15,57,1.000000,983.125439,983.125439,402.923516,0.936906,1049.331569,430.057395,1.000000,983.125439,402.923516


# CLV gemiddelde toevoegen met churn per 6 maanden en jaar

In [296]:
# Toevoegen van gemiddelde CLV aan de data set bij CLV berekend met churn per 6 maanden
df_clv_cum_6M = df_transacties[['CLV_churn_6M', 'CLV_margin_churn_6M']].copy()
df_clv_cum_6M.dropna(inplace=True)
df_clv_cum['CLV_AVG_6M'] = df_clv_cum_6M['CLV_churn_6M'].cumsum().div(df_clv_cum['Index'])
df_clv_cum['CLV_margin_AVG_6M'] = df_clv_cum_6M['CLV_margin_churn_6M'].cumsum().div(df_clv_cum['Index'])
df_clv_cum.head()

,CLV,CLV_margin,Index,CLV_AVG,CLV_margin_AVG,CLV_AVG_6M,CLV_margin_AVG_6M
Orderdate,,,,,,,
2014-04-30,840.000000,344.265075,1,840.000000,344.265075,840.000000,344.265075
2014-07-31,235.000000,96.312253,2,537.500000,220.288664,537.500000,220.288664
2014-09-30,219.000000,89.754823,3,431.333333,176.777384,431.333333,176.777384
2014-10-31,564.000000,231.149408,4,464.500000,190.370390,464.500000,190.370390
2014-11-30,675.251111,276.744494,5,506.650222,207.645211,515.744843,211.372544


In [297]:
# Toevoegen van gemiddelde CLV aan de data set bij CLV berekend met churn per jaar
df_clv_cum_Y = df_transacties[['CLV_churn_Y', 'CLV_margin_churn_Y']].copy()
df_clv_cum_Y.dropna(inplace=True)
df_clv_cum['CLV_AVG_Y'] = df_clv_cum_Y['CLV_churn_Y'].cumsum().div(df_clv_cum['Index'])
df_clv_cum['CLV_margin_AVG_Y'] = df_clv_cum_Y['CLV_margin_churn_Y'].cumsum().div(df_clv_cum['Index'])
df_clv_cum.head()

,CLV,CLV_margin,Index,CLV_AVG,CLV_margin_AVG,CLV_AVG_6M,CLV_margin_AVG_6M,CLV_AVG_Y,CLV_margin_AVG_Y
Orderdate,,,,,,,,,
2014-04-30,840.000000,344.265075,1,840.000000,344.265075,840.000000,344.265075,840.000000,344.265075
2014-07-31,235.000000,96.312253,2,537.500000,220.288664,537.500000,220.288664,537.500000,220.288664
2014-09-30,219.000000,89.754823,3,431.333333,176.777384,431.333333,176.777384,431.333333,176.777384
2014-10-31,564.000000,231.149408,4,464.500000,190.370390,464.500000,190.370390,464.500000,190.370390
2014-11-30,675.251111,276.744494,5,506.650222,207.645211,515.744843,211.372544,506.650222,207.645211


In [298]:
# Dataset overhouden met alle gemiddeldes
# CLV_AVG is churn berekend per maand, 6M churn berekend over 6 maanden, Y is churn berekend over een jaar
# Margin is steeds berekend over de gehele dataset en zelfde marge is gebruikt in alle berekeningen (kan eventueel nog per maand, 6 maanden, jaar etc. berekend worden)
df_clv_avg = df_clv_cum[['CLV_AVG', 'CLV_margin_AVG', 'CLV_AVG_6M', 'CLV_margin_AVG_6M', 'CLV_AVG_Y', 'CLV_margin_AVG_Y']].copy()
df_clv_avg.head(10)

,CLV_AVG,CLV_margin_AVG,CLV_AVG_6M,CLV_margin_AVG_6M,CLV_AVG_Y,CLV_margin_AVG_Y
Orderdate,,,,,,
2014-04-30,840.000000,344.265075,840.000000,344.265075,840.000000,344.265075
2014-07-31,537.500000,220.288664,537.500000,220.288664,537.500000,220.288664
2014-09-30,431.333333,176.777384,431.333333,176.777384,431.333333,176.777384
2014-10-31,464.500000,190.370390,464.500000,190.370390,464.500000,190.370390
2014-11-30,506.650222,207.645211,515.744843,211.372544,506.650222,207.645211
2014-12-31,586.062758,240.191595,604.675964,247.820019,586.062758,240.191595
2015-01-31,581.997481,238.525484,600.352780,246.048208,589.166136,241.463481
2015-02-28,574.893026,235.613798,593.158685,243.099785,587.312373,240.703736
2015-03-31,570.484670,233.807081,588.835552,241.327995,587.216750,240.664546


In [ ]:
df_clv_avg.to_excel('Gemiddeldes CLV per maand.xlsx')